In [8]:
import torchvision

resnet = torchvision.models.resnet18(pretrained = True)
print(resnet)

ModuleNotFoundError: No module named 'torchvision'

In [6]:
from tensorflow.keras.applications import ResNet50
import tensorflow as tf

from data_proccessing import image_size, labels

rnet = ResNet50(
        input_shape=(image_size, image_size, 3),
        weights='imagenet',
        include_top=False
    )
rnet.trainable = True

rnet_model = tf.keras.Sequential([
    rnet,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(len(labels), activation='sigmoid')])

rnet_model.compile(
    optimizer='adam',
    loss = 'binary_crossentropy',
    metrics=['AUC','Precision','Recall'])

In [7]:
from sklearn.metrics import roc_curve, auc, roc_auc_score, accuracy_score, average_precision_score
from data_proccessing import train_gen, valid_gen, X_val, y_val

resnet1 = rnet_model.fit(train_gen,
              validation_data = valid_gen,
              epochs = 10,) # 95 min

y_pred = rnet_model.predict(X_val)

Epoch 1/10
145/145 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - AUC: 0.6036 - Precision: 0.1799 - Recall: 0.0444 - loss: 0.3755

/Users/abigail/Documents/adl/venv/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


145/145 ━━━━━━━━━━━━━━━━━━━━ 856s 6s/step - AUC: 0.6039 - Precision: 0.1802 - Recall: 0.0442 - loss: 0.3753 - val_AUC: 0.4910 - val_Precision: 0.1766 - val_Recall: 0.1056 - val_loss: 0.4725
Epoch 2/10
145/145 ━━━━━━━━━━━━━━━━━━━━ 581s 4s/step - AUC: 0.6745 - Precision: 0.4233 - Recall: 0.0109 - loss: 0.3282 - val_AUC: 0.5945 - val_Precision: 0.0000e+00 - val_Recall: 0.0000e+00 - val_loss: 0.3540
Epoch 3/10
145/145 ━━━━━━━━━━━━━━━━━━━━ 552s 4s/step - AUC: 0.7043 - Precision: 0.5266 - Recall: 0.0227 - loss: 0.3194 - val_AUC: 0.5819 - val_Precision: 0.0000e+00 - val_Recall: 0.0000e+00 - val_loss: 0.3962
Epoch 4/10
145/145 ━━━━━━━━━━━━━━━━━━━━ 532s 4s/step - AUC: 0.7153 - Precision: 0.5255 - Recall: 0.0318 - loss: 0.3167 - val_AUC: 0.6022 - val_Precision: 0.0000e+00 - val_Recall: 0.0000e+00 - val_loss: 0.3570
Epoch 5/10
145/145 ━━━━━━━━━━━━━━━━━━━━ 540s 4s/step - AUC: 0.7261 - Precision: 0.5471 - Recall: 0.0370 - loss: 0.3135 - val_AUC: 0.6108 - val_Precision: 0.0000e+00 - val_Recall: 0.00

In [9]:
import numpy as np

for label, p_count, t_count in zip(labels,
                                     100 * np.mean(y_pred, 0),
                                     100 * np.mean(y_val, 0)):
    print('%s: actual: %2.2f%%, predicted: %2.2f%%' % (label, t_count, p_count))


Atelectasis: actual: 16.54%, predicted: 27.49%
Cardiomegaly: actual: 9.09%, predicted: 7.35%
Consolidation: actual: 9.78%, predicted: 18.82%
Edema: actual: 9.44%, predicted: 14.92%
Effusion: actual: 17.66%, predicted: 23.91%
Emphysema: actual: 9.09%, predicted: 7.34%
Fibrosis: actual: 8.92%, predicted: 11.49%
Hernia: actual: 3.98%, predicted: 6.15%
Infiltration: actual: 25.54%, predicted: 39.01%
Mass: actual: 9.96%, predicted: 20.55%
No Finding: actual: 8.66%, predicted: 14.81%
Nodule: actual: 10.39%, predicted: 17.82%
Pleural_Thickening: actual: 8.40%, predicted: 9.30%
Pneumonia: actual: 8.92%, predicted: 15.26%
Pneumothorax: actual: 10.82%, predicted: 10.42%


In [14]:
val_metrics = rnet_model.evaluate(valid_gen)
print("Validation Metrics:", val_metrics)


1/1 ━━━━━━━━━━━━━━━━━━━━ 42s 42s/step - AUC: 0.6222 - Precision: 0.3200 - Recall: 0.0041 - loss: 0.3530
Validation Metrics: [0.3529972434043884, 0.6221532821655273, 0.3199999928474426, 0.004142931196838617]


In [16]:
loss, sparse_categorical_accuracy, val_loss, val_sparse_categorical_accuracy = val_metrics
print("Loss: ", loss)
print("Sparse_Categorical_Accuracy: ", sparse_categorical_accuracy)

Loss:  0.3529972434043884
Sparse_Categorical_Accuracy:  0.6221532821655273


In [17]:
rnet_model.save('model/resnet.keras')

In [11]:
import matplotlib.pyplot as plt

plt.plot(resnet1.history['AUC']) 
plt.plot(resnet1.history['val_AUC']) 
plt.title('model auc')
plt.ylabel('auc')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

ModuleNotFoundError: No module named 'matplotlib'